# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import _data_utils
import _script_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))

In [6]:

# df = utils.read_data_all()
proj = _data_utils.add_new_codes(proj)
    
    ## function that adds known agency name to df 
df = _script_utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [7]:
df.sample(3)

,unnamed:_0,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
9381,NaN,2023-01-23,ZS3E,NaN,S074071,0813000047S,"IN RIVERSIDE COUNTY NEAR LAKE ELSINORE FROM ORANGE/RIVERSIDE COUNTY LINE TO MONTE VISTA STREET WIDEN EXISTING LANES TO PROVIDE 12' LANES, 4' SHOULDER",65,Cong Dist 42,Active,"ON STATE ROUTE: 74. IN RIVERSIDE COUNTY NEAR LAKE ELSINORE FROM ORANGE/RIVERSIDE COUNTY LINE TO MONTE VISTA STREET WIDEN EXISTING LANES TO PROVIDE 12' LANES, 4' SHOULDERS AND GROUND-IN RUMBLE STRIPS CENTERLINE AND SHOULDERS",17,Construction Engineering,7868000.00,5223600.00,S SCAG,NaN,California,8.0,Riverside County,NaN,NaN
906,NaN,2022-03-10,M001,NaN,5030056,0412000201L,SACRAMENTO STREET OH IN THE CITY OF VALLEJO BETWEEN ILLINOIS ST. AND FARRAGUT AVE. SEISMIC RETROFIT - REPLACE BRIDGE,95,Cong Dist 5,Active,SACRAMENTO STREET OH IN THE CITY OF VALLEJO BETWEEN ILLINOIS ST. AND FARRAGUT AVE. SEISMIC RETROFIT - REPLACE BRIDGE,11,Bridge Replacement - No Added Capacity,1287312.23,1121686.54,L5030MTC,5030.0,Vallejo,4.0,Solano County,NON-RTPA,Metropolitan Transportation Commission
1978,NaN,2022-05-31,Z905,NaN,5380028,0813000056L,GREEN TREE BOULEVARD FROM HESPERIA ROAD TO RIDGECREST BOULEVARD EXTENSION OF 4-LANE ROAD WITH NEW OVERHEAD,71,Cong Dist 8,Active,GREEN TREE BOULEVARD FROM HESPERIA ROAD TO RIDGECREST BOULEVARD EXTENSION OF 4-LANE ROAD WITH NEW OVERHEAD,1,New Construction Roadway,4764591.00,4218092.00,L5380SCAG,5380.0,Victorville,8.0,San Bernardino County,San Bernardino Associated Governments,Southern California Association Of Governments


## Test & Export

In [8]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

In [9]:
agg.sample(10)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
2214,2022-06-29,5938264,Stanislaus County,L5938STANCOG,Z400,nan,1020000148L,44,Other,THROUGHOUT STANISLAUS COUNTY TRANSIT FARE SUBSIDY(TFS) START/MAX EMPLOYEE RIDE PROGRAM,84989.00,Cong Dist 10,10.0,99,Stanislaus County,Stanislaus County,5938.0,Stanislaus Council of Governments,Stanislaus Council Of Governments,Project in Stanislaus County,"This project is part of the nan Program, and recieved $ 84989.0. This project will Project in Stanislaus County"
3934,2022-10-25,5057019,Berkeley,L5057MTC,H400,nan,04072124L,16,Right of Way,SAN PABLO AVE. CORRIDOR IN CITY OF BERKELEY BICYCLE RELATED - OTHER,1034000.00,Cong Dist 13,4.0,1,Alameda County,Alameda County,5057.0,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Preliminary Engineering Projects in Berkeley,"This project is part of the nan Program, and recieved $ 1034000.0. This project will Preliminary Engineering Projects in Berkeley"
5938,2023-03-08,5241065,Ceres,L5241STANCOG,Z40E,nan,1017000098L,24,Traffic Management/Engineering - HOV,INTERSECTION OF SERVICE ROAD AND MORGAN ROAD TRAFFIC SIGNAL AT MORGAN AND SERVICE ROAD,304150.00,Cong Dist 10,10.0,99,Stanislaus County,Stanislaus County,5241.0,Stanislaus Council of Governments,Stanislaus Council Of Governments,Traffic Signals in Ceres,"This project is part of the nan Program, and recieved $ 304150.0. This project will Traffic Signals in Ceres"
1819,2022-06-06,5060364,Fresno,L5060COFCG,Z0E3,nan,0619000170L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,HERNDON CANAL METRO TRAIL FROM SHIELDS TO MCKINLEY AVENUES. CONSTRUCTION OF PEDESTRIAN AND CYCLE TRAIL ALONG HERNDON CANAL WITH BENCHING AND LANDSCAP,3415300.00,Cong Dist 16,6.0,19,Fresno County,Fresno County,5060.0,Council of Fresno County Governments,Council Of Fresno County Goverments,Construct Pedestrian Safety Improvements in Fresno,"This project is part of the nan Program, and recieved $ 3415300.0. This project will Construct Pedestrian Safety Improvements in Fresno"
5021,2023-01-09,38M0002,Santa Barbara County,L5951SBCAG,ER12,nan,0518000171L,4|5|15|17,4R - No Added Capacity|4R - Maintenance Resurfacing|Preliminary Engineering|Construction Engineering,"NORTH JAMESON LANE, FROM OLIVE MILL RD TO SHEFFIELD RD INCLUDING BR 51C-341,51C-342,51C-343 IN THE VILLAGE OF MONTECITO REPAIRS TO BRIDGE RAILINGS, C",390404.46,Cong Dist 24,5.0,83,Santa Barbara County,Santa Barbara County,5951.0,Santa Barbara County Association of Governments,Santa Barbara County Association Of Governments,Maintenance Resurfacing in Santa Barbara County,"This project is part of the nan Program, and recieved $ 390404.46. This project will Maintenance Resurfacing in Santa Barbara County"
3656,2022-09-22,0010204,California,S TAHOE,Y450,Metropolitan Planning Program,0022000349S,18,Planning,OWP 22/23 TAHOE REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR TAHOE REGIONAL PLANNING AGENCY (TRPA) OVERALL WORK PROGRAM (OWP) FOR FISCAL YEAR,753713.00,Cong Dist 0,nan,999,Statewide,Statewide,nan,None,None,Planning and Research Statewide,"This project is part of the Metropolitan Planning Program Program, and recieved $ 753713.0. This project will Planning and Research Statewide"
2594,2022-07-26,5129070,Oxnard,L5129SCAG,L1CE,nan,0712000045L,15,Preliminary Engineering,"CHANNEL ISLAND BLVD OVER EDISON CANAL, 0.2 MILES EAST OF HARBOR BLVD, 52C0095 BRIDGE REHABILITATION (NO ADDED LANE CAPACITY)",56826.83,Cong Dist 26,7.0,111,Ventura County,Ventura County,5129.0,Ventura County Transportation Commission,Southern California Association Of Governments,Bridge Rehabilitation in Oxnard,

In [10]:
sorted(list(agg.project_title_new))

[' Added Roadway Capacity in Industry',
 ' Added Roadway Capacity in Kern County',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Madera County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Murrieta',
 ' Added Roadway Capacity in Murrieta',
 ' Added Roadway 

In [11]:
len(agg)

5945

In [12]:
agg = _script_utils.title_column_names(agg)

In [13]:
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_agg.csv")

In [14]:
# ###test full. PASS title, no concat and 1241 rows
# full = _script_utils.get_clean_data(df, full_or_agg = 'full')

In [16]:
# full>>filter(_["Recipient Project Number"]=="0118000055L")

In [17]:
# full = _script_utils.title_column_names(full)

In [ ]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_all.csv")